In [35]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input, GlobalAveragePooling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.utils import to_categorical

# ========================================================
# 1. Configuration Settings
# ========================================================
# Set random seeds for reproducibility
RANDOM_SEED = 42
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
import tensorflow as tf
tf.random.set_seed(RANDOM_SEED)

# Image parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-4

# Paths
train_csv_path = 'train.csv'
test_csv_path = 'test.csv'
base_image_dir = './train/train'
test_directory = './test/test'
model_path = 'skin_condition_model_improved.keras'  # Changed from .h5 to .keras

# ========================================================
# 2. Data Augmentation Functions
# ========================================================
def augment_image(img):
    """Apply random augmentations to a single image"""
    # Convert PIL Image to NumPy array if needed
    if isinstance(img, Image.Image):
        img_array = np.array(img)
    else:
        img_array = img.copy()
    
    # Random flip
    if random.random() > 0.5:
        img_array = np.fliplr(img_array)
    
    # Random rotation (convert back to PIL for rotation)
    if random.random() > 0.5:
        angle = random.uniform(-20, 20)
        img_pil = Image.fromarray(img_array.astype('uint8'))
        img_pil = img_pil.rotate(angle, resample=Image.BILINEAR, expand=False)
        img_array = np.array(img_pil)
    
    # Random brightness and contrast (convert to PIL)
    if random.random() > 0.5:
        img_pil = Image.fromarray(img_array.astype('uint8'))
        enhancer = ImageEnhance.Brightness(img_pil)
        factor = random.uniform(0.8, 1.2)
        img_pil = enhancer.enhance(factor)
        
        enhancer = ImageEnhance.Contrast(img_pil)
        factor = random.uniform(0.8, 1.2)
        img_pil = enhancer.enhance(factor)
        
        img_array = np.array(img_pil)
        
    return img_array

# ========================================================
# 3. Load and Clean CSV Data
# ========================================================
print("Loading and cleaning CSV data...")
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

# Add '.jpg' extension to md5hash
train_df['md5hash'] = train_df['md5hash'].astype(str) + '.jpg'
test_df['md5hash'] = test_df['md5hash'].astype(str) + '.jpg'

# Combine label and md5hash to form the file path (e.g. "acne/xyz.jpg")
train_df['file_path'] = train_df['label'] + '/' + train_df['md5hash']

# Replace ddi_scale with fitzpatrick_centaur
train_df['fitzpatrick_scale'] = train_df['fitzpatrick_centaur']
test_df['fitzpatrick_scale'] = test_df['fitzpatrick_centaur']
train_df.drop(columns=['ddi_scale'], inplace=True)
test_df.drop(columns=['ddi_scale'], inplace=True)

# Remove rows with wrongly labelled data
train_df = train_df[train_df['qc'] != '3 Wrongly labelled']
test_df = test_df[test_df['qc'] != '3 Wrongly labelled']

# Encode the label column numerically
label_encoder = LabelEncoder()
train_df['label_numerical'] = label_encoder.fit_transform(train_df['label'])
label_mapping = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))
inverse_label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))

print(f"Number of classes: {len(label_mapping)}")
print(f"Label mapping: {label_mapping}")

# Encode fitzpatrick scale as numerical to use as additional feature
train_df['fitzpatrick_numerical'] = train_df['fitzpatrick_scale'].fillna(0).astype(int)
test_df['fitzpatrick_numerical'] = test_df['fitzpatrick_scale'].fillna(0).astype(int)

# Check class distribution
class_counts = train_df['label_numerical'].value_counts()
print("Class distribution:\n", class_counts)

# ========================================================
# 4. Build the Dataset in Memory
# ========================================================
print("Loading and processing training images...")
all_images = []
all_labels = []
all_fitzpatrick = []

for idx, row in train_df.iterrows():
    file_path = os.path.join(base_image_dir, row['file_path'])
    
    # Load and resize image
    if os.path.exists(file_path):
        try:
            with Image.open(file_path) as img:
                img = img.resize(IMAGE_SIZE)
                img = img.convert('RGB')
                img_array = np.array(img, dtype=np.float32)
                
                # Normalize pixel values to [0,1]
                img_array = img_array / 255.0
                
                all_images.append(img_array)
                all_labels.append(row['label_numerical'])
                all_fitzpatrick.append(row['fitzpatrick_numerical'])
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
    else:
        print(f"Warning: File not found {file_path}")

# Convert lists to NumPy arrays
X = np.array(all_images, dtype=np.float32)
y = np.array(all_labels, dtype=np.int32)
fitzpatrick_data = np.array(all_fitzpatrick, dtype=np.float32)

# Convert labels to one-hot encoding
y_onehot = to_categorical(y, num_classes=len(label_mapping))

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Shape of fitzpatrick_data:", fitzpatrick_data.shape)

# ========================================================
# 5. Split into Training and Validation Sets
# ========================================================
X_train, X_val, y_train, y_val, fitz_train, fitz_val = train_test_split(
    X, y, fitzpatrick_data, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

# One-hot encoded versions for categorical loss
y_train_onehot = to_categorical(y_train, num_classes=len(label_mapping))
y_val_onehot = to_categorical(y_val, num_classes=len(label_mapping))

# ========================================================
# 6. Data Augmentation Setup
# ========================================================
print("Setting up data augmentation...")
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Calculate class weights to handle imbalance
class_weight_dict = {i: np.float32(weight) for i, weight in enumerate(class_weights)}

print("Class weights:", class_weight_dict)

BATCH_SIZE = 16  # Reduced from 32

# 5. Try training without the generator first to isolate the issue
try:
    # First attempt without generator to isolate the issue
    simple_history = model.fit(
        X_train, 
        y_train_onehot,
        batch_size=BATCH_SIZE,
        epochs=2,  # Just a couple epochs to test
        validation_data=(X_val, y_val_onehot),
        class_weight=class_weight_dict,
        verbose=1
    )
    print("Direct training successful, now trying with generator...")
    
    # Now try with generator
    train_generator = datagen.flow(
        X_train, 
        y_train_onehot,
        batch_size=BATCH_SIZE
    )
    
    history = model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val_onehot),
        callbacks=callbacks,
        class_weight=class_weight_dict,
        verbose=1
    )
except Exception as e:
    print(f"Error during training: {e}")
    print("Trying alternative approach...")
    
    # If still failing, try without class weights
    try:
        history = model.fit(
            X_train, 
            y_train_onehot,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(X_val, y_val_onehot),
            callbacks=callbacks,
            verbose=1
        )
    except Exception as e:
        print(f"Training still failing: {e}")
        
# ========================================================
# 7. Build the Model without Multiple Inputs
# ========================================================
print("Building simplified model with transfer learning...")

def build_model(input_shape, num_classes):
    # Load pre-trained EfficientNetB3 without top layers
    base_model = EfficientNetB3(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Freeze early layers
    for layer in base_model.layers[:100]:
        layer.trainable = False
    
    # Image input
    img_input = Input(shape=input_shape)
    x = base_model(img_input)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    # Create model
    model = Model(inputs=img_input, outputs=output)
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Build model
model = build_model(
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    num_classes=len(label_mapping)
)

model.summary()

# ========================================================
# 8. Train with Callbacks
# ========================================================
print("Training model...")

# Setup callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss', 
        patience=7,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    ),
    ModelCheckpoint(
        model_path,
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Prepare data for training with datagen
train_generator = datagen.flow(
    X_train, 
    y_train_onehot,
    batch_size=BATCH_SIZE
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val_onehot),
    callbacks=callbacks,
    class_weight=class_weight_dict,  # This works with standard ImageDataGenerator
    verbose=1
)

print("Model trained and saved as", model_path)

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.savefig('training_history.png')
print("Training history saved as training_history.png")

# ========================================================
# 9. Load Best Model and Make Predictions
# ========================================================
print("Loading best model for predictions...")
best_model = tf.keras.models.load_model(model_path)

# ========================================================
# 10. Process Test Data
# ========================================================
print("Processing test data...")
# Get list of all image files in the test directory
image_files = [file for file in os.listdir(test_directory) if file.endswith('.jpg')]
print(f"Total image files in test directory: {len(image_files)}")

# Find any discrepancies between directory and CSV
test_file_set = set(test_df['md5hash'])
directory_file_set = set(image_files)
missing_files = directory_file_set - test_file_set
extra_files = test_file_set - directory_file_set

print(f"Files in directory but not in CSV: {missing_files}")
print(f"Files in CSV but not in directory: {extra_files}")

# Process all images in the test directory
test_images = []
test_md5hashes = []

# First process entries in the test_df
for idx, row in test_df.iterrows():
    file_path = os.path.join(test_directory, row['md5hash'])
    if os.path.exists(file_path):
        try:
            with Image.open(file_path) as img:
                img = img.resize(IMAGE_SIZE)
                img = img.convert('RGB')
                img_array = np.array(img, dtype=np.float32)
                img_array /= 255.0
                
                test_images.append(img_array)
                test_md5hashes.append(row['md5hash'].replace('.jpg', ''))
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
    else:
        print(f"Warning: Test file not found {file_path}")

# Then process any additional files found in the directory but not in the CSV
for filename in missing_files:
    file_path = os.path.join(test_directory, filename)
    try:
        with Image.open(file_path) as img:
            img = img.resize(IMAGE_SIZE)
            img = img.convert('RGB')
            img_array = np.array(img, dtype=np.float32)
            img_array /= 255.0
            
            test_images.append(img_array)
            test_md5hashes.append(filename.replace('.jpg', ''))
    except Exception as e:
        print(f"Error processing additional file {file_path}: {e}")

X_test = np.array(test_images, dtype=np.float32)
print(f"Final processed test images: {len(test_images)}")

# ========================================================
# 11. Test Time Augmentation
# ========================================================
print("Generating predictions with Test Time Augmentation...")

def tta_predict(model, img_array, num_augments=5):
    """Test time augmentation for more robust predictions"""
    preds = []
    
    # Original prediction
    pred = model.predict(np.expand_dims(img_array, axis=0))[0]
    preds.append(pred)
    
    # Get predictions with augmentations
    for _ in range(num_augments):
        aug_img = augment_image(img_array)
        aug_pred = model.predict(np.expand_dims(aug_img, axis=0))[0]
        preds.append(aug_pred)
    
    # Average predictions
    avg_pred = np.mean(preds, axis=0)
    return avg_pred

# Apply TTA to each test image
all_predictions = []
for i in range(len(X_test)):
    if i % 100 == 0:
        print(f"Processing test image {i}/{len(X_test)}")
    
    pred = tta_predict(best_model, X_test[i])
    all_predictions.append(pred)

predictions = np.array(all_predictions)
predicted_classes = np.argmax(predictions, axis=1)

# Convert numerical labels back to original class names
predicted_labels = [label_mapping[cls] for cls in predicted_classes]

# Create submission dataframe
submission_df = pd.DataFrame({
    'md5hash': test_md5hashes,
    'label': predicted_labels
})

# Save to CSV without index
submission_df.to_csv('submission.csv', index=False)
print("Predictions saved to submission.csv")

# ========================================================
# 12. Evaluate Model Performance
# ========================================================
print("Evaluating model performance...")
# Evaluate on validation set
val_preds = best_model.predict(X_val)
val_pred_classes = np.argmax(val_preds, axis=1)

# Calculate accuracy
val_accuracy = np.mean(val_pred_classes == y_val)
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Generate classification report
from sklearn.metrics import classification_report, confusion_matrix
print("\nClassification Report:")
print(classification_report(y_val, val_pred_classes, target_names=list(label_mapping.values())))

# Create confusion matrix
conf_matrix = confusion_matrix(y_val, val_pred_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nTraining and prediction complete!")

Loading and cleaning CSV data...
Number of classes: 21
Label mapping: {0: 'acne', 1: 'acne-vulgaris', 2: 'actinic-keratosis', 3: 'basal-cell-carcinoma', 4: 'basal-cell-carcinoma-morpheiform', 5: 'dermatofibroma', 6: 'dermatomyositis', 7: 'dyshidrotic-eczema', 8: 'eczema', 9: 'epidermal-nevus', 10: 'folliculitis', 11: 'kaposi-sarcoma', 12: 'keloid', 13: 'malignant-melanoma', 14: 'melanoma', 15: 'mycosis-fungoides', 16: 'prurigo-nodularis', 17: 'pyogenic-granuloma', 18: 'seborrheic-keratosis', 19: 'squamous-cell-carcinoma', 20: 'superficial-spreading-melanoma-ssm'}
Class distribution:
 label_numerical
19    407
3     328
10    237
1     234
14    181
8     143
15    127
0     127
2     122
16    119
11    109
12    109
6     106
20     83
17     79
13     78
9      64
7      58
5      55
18     47
4      43
Name: count, dtype: int64
Loading and processing training images...
Shape of X: (2856, 224, 224, 3)
Shape of y: (2856,)
Shape of fitzpatrick_data: (2856,)
Setting up data augmentation

c:\Users\juesh\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
Error during training: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32, tf.float64, tf.float32), but the yielded element was (array([[[[0.7254902 , 0.7411765 , 0.7764706 ],
         [0.7254902 , 0.7411765 , 0.7764706 ],
         [0.7254902 , 0.7411765 , 0.7764706 ],
         ...,
         [0.7254902 , 0.7411765 , 0.7764706 ],
         [0.7254902 , 0.7411765 , 0.7764706 ],
         [0.7254902 , 0.7411765 , 0.7764706 ]],

        [[0.7254902 , 0.7411765 , 0.7764706 ],
         [0.7254902 , 0.7411765 , 0.7764706 ],
         [0.7254902 , 0.7411765 , 0.7764706 ],
         ...,
         [0.7254902 , 0.7411765 , 0.7764706 ],
         [0.7254902 , 0.7411765 , 0.7764706 ],
         [0.7254902 , 0.7411765 , 0.7764706 ]],

        [[0.7254902 , 0.7411765 , 0.7764706 ],
         [0.7254902 , 0.7411765 ,

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_24 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb3 (Functional)     │ (None, 7, 7, 1536)     │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_12     │ (None, 1536)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 1536)           │         6,144 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 512)            │       786,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 21)             │         5,397 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,716,420 (44.69 MB)

 Trainable params: 11,504,219 (43.89 MB)

 Non-trainable params: 212,201 (828.91 KB)

Training model...
Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32, tf.float64, tf.float32), but the yielded element was (array([[[[0.08128925, 0.06952454, 0.11266179],
         [0.08197479, 0.07021008, 0.11334734],
         [0.08235294, 0.07058824, 0.11372549],
         ...,
         [0.02068717, 0.01676561, 0.04529592],
         [0.0255673 , 0.02062679, 0.05607511],
         [0.03233111, 0.02448797, 0.07346391]],

        [[0.07843138, 0.06666667, 0.10980392],
         [0.07843138, 0.06666667, 0.10980392],
         [0.07879915, 0.06703445, 0.1101717 ],
         ...,
         [0.02137272, 0.01745115, 0.04666701],
         [0.02693839, 0.02131233, 0.05950284],
         [0.03301665, 0.02517352, 0.07552055]],

        [[0.07843138, 0.06666667, 0.10980392],
         [0.07843138, 0.06666667, 0.10980392]

In [26]:
print(f"Number of images found in the test set: {len(test_images)}")
print(f"Expected number of images in the test set: {len(test_df)}")

import os

# Define the path to the test directory
test_directory = './test/test'

# Get a list of all files in the directory
all_files = os.listdir(test_directory)

# Filter the list to include only image files (e.g., '.jpg' files)
image_files = [file for file in all_files if file.endswith('.jpg')]

# Print the number of image files
print(f"Number of image files in '{test_directory}': {len(image_files)}")




Number of images found in the test set: 1226
Expected number of images in the test set: 1226
Number of image files in './test/test': 1227


Missing image files: []


Duplicate files found: []
